In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV

In [2]:
# #PATH

path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_file = "F:/178/Tugas Akhir/dataset/clips_45sec_wav/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"

In [4]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values
    
def predict(data,values,test,true,clf):
    svr = clf
    svr.fit(data,values)
    a = svr.predict(test)
    df = pd.DataFrame(data=[a,true])
    
    return a,df

def score(a,true):
    r2 = r2_score(true, a)
    
    return r2

def gridsearch(data,true):
    svr = SVR(kernel='rbf')
    Cs = [1e-4,1e-3,1e-2,1e-1,1,1e2,1e3,1e4,1e5]
    gammas = [1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1e-10,1e-11,1e-12,1e-13,1e-14]
    parameters = {'C':Cs,'gamma':gammas}

    clf = GridSearchCV(svr, parameters, cv=10, scoring='r2')
    clf.fit(data,true)

    return clf.best_params_ , clf.best_score_

def regresion(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

# CASE 1

In [5]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

svr_case1_a = SVR(kernel='rbf',C=1e2,gamma =1e-6) #init Arousal
svr_case1_v = SVR(kernel='rbf',C=1,gamma =1e-5) #init Valence

#Predict Valence
a_v,df_v = predict(train_data_v,train_values_v,test_data_v,test_values_v,svr_case1_v)
#Predict Arousal
a_a,df_a = predict(train_data_a,train_values_a,test_data_a,test_values_a,svr_case1_a)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

(744, 26)


In [6]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [7]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 100.0, 'gamma': 1e-06}
parameter Valence:	 {'C': 1, 'gamma': 1e-05}


AROUSAL

In [8]:
df_a = df_a.round(2)
display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,4.9,5.7,7.05,3.55,3.45,4.38,2.35,6.05,3.53,5.04,...,6.78,5.07,4.58,4.65,4.93,4.77,5.79,6.24,4.33,3.02
1,6.2,5.9,6.40,3.10,3.50,3.60,3.00,5.80,3.60,6.20,...,7.00,5.00,3.70,5.80,4.70,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.65


VALENCE

In [9]:
df_v = df_v.round(2)
display(df_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.83,5.73,5.67,4.4,4.61,5.05,4.01,5.6,4.18,5.44,...,5.75,5.54,4.75,5.31,5.47,5.2,5.44,5.56,4.51,4.01
1,2.70,6.40,6.00,2.7,5.00,3.20,4.60,4.3,4.10,6.70,...,6.60,4.80,2.50,5.60,3.80,4.4,6.50,6.50,4.00,3.50


R2 Score Valence: 0.24


# REGRESSION AND GRIDSEARCHCV

In [10]:
svr_a = SVR(kernel='rbf',C=1e2,gamma =1e-6)
svr_v = SVR(kernel='rbf',C=1,gamma =1e-5)

scores_a = regresion(svr_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_v,data,valence_mean) #Arousal Regression

In [11]:
print("R2 Score Arousal:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Arousal:	0.56 (+/- 0.09)
R2 Score Valence:	0.10 (+/- 0.19)


# CASE 2

In [12]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

svr_case1_a = SVR(kernel='rbf',C=1e4,gamma =1e-9) #init Arousal
svr_case1_v = SVR(kernel='rbf',C=1e5,gamma =1e-10) #init Valence

#Predict Valence
a_v,df_v = predict(train_data_v,train_values_v,test_data_v,test_values_v,svr_case1_v)
#Predict Arousal
a_a,df_a = predict(train_data_a,train_values_a,test_data_a,test_values_a,svr_case1_a)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_vv = score(a_v,test_values_v) #Score Valence

(744, 34)


In [15]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [16]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 10000.0, 'gamma': 1e-09}
parameter Valence:	 {'C': 100000.0, 'gamma': 1e-10}


AROUSAL

In [17]:
df_a = df_a.round(2)
display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.28,5.49,6.94,3.49,3.49,4.38,2.68,5.91,3.29,4.99,...,6.6,5.16,4.79,4.78,5.24,4.64,5.74,6.26,4.18,3.08
1,6.20,5.90,6.40,3.10,3.50,3.60,3.00,5.80,3.60,6.20,...,7.0,5.00,3.70,5.80,4.70,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.65


VALENCE

In [18]:
df_v = df_v.round(2)
display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,6.08,5.64,6.19,4.35,4.73,4.91,4.01,5.65,4.16,5.35,...,6.22,5.79,4.72,5.04,5.57,5.06,5.41,5.33,4.29,4.01
1,2.70,6.40,6.00,2.70,5.00,3.20,4.60,4.30,4.10,6.70,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.24


# REGRESSION AND GRIDSEARCHCV

In [19]:
svr_a = SVR(kernel='rbf',C=1e4,gamma =1e-9)
svr_v = SVR(kernel='rbf',C=1e5,gamma =1e-10)

scores_a = regresion(svr_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_v,data,valence_mean) #Arousal Regression

In [20]:
print("R2 Score Arousal:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Arousal:	0.55 (+/- 0.09)
R2 Score Valence:	0.09 (+/- 0.21)


# CASE 3

In [24]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

svr_case1_a = SVR(kernel='rbf',C=1e2,gamma =1e-8) #init Arousal
svr_case1_v = SVR(kernel='rbf',C=1e2,gamma =1e-9) #init Valence

#Predict Valence
a_v,df_v = predict(train_data_v,train_values_v,test_data_v,test_values_v,svr_case1_v)
#Predict Arousal
a_a,df_a = predict(train_data_a,train_values_a,test_data_a,test_values_a,svr_case1_a)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

(744, 38)


In [22]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [23]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 100.0, 'gamma': 1e-08}
parameter Valence:	 {'C': 100.0, 'gamma': 1e-09}


AROUSAL

In [25]:
df_a = df_a.round(2)
display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.44,4.73,6.69,3.48,3.5,4.99,2.89,6.03,3.29,5.16,...,6.4,5.26,4.96,4.77,5.4,4.33,5.75,6.45,3.91,2.95
1,6.20,5.90,6.40,3.10,3.5,3.60,3.00,5.80,3.60,6.20,...,7.0,5.00,3.70,5.80,4.7,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.58


VALENCE

In [26]:
df_v = df_v.round(2)
display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.92,5.19,5.81,4.35,4.71,4.8,4.03,5.48,4.2,5.3,...,6.03,5.73,4.97,4.81,5.79,4.65,5.29,5.55,4.37,4.04
1,2.70,6.40,6.00,2.70,5.00,3.2,4.60,4.30,4.1,6.7,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.18


# REGRESSION AND GRIDSEARCHCV

In [29]:
svr_a = SVR(kernel='rbf',C=1e2,gamma =1e-8)
svr_v = SVR(kernel='rbf',C=1e2,gamma =1e-9)

scores_a = regresion(svr_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_v,data,valence_mean) #Arousal Regression

In [30]:
print("R2 Score Arousal:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Arousal:	0.51 (+/- 0.06)
R2 Score Valence:	0.06 (+/- 0.17)


# CASE 3 V2

In [35]:
datafitur =  "case3_v2" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

svr_case1_a = SVR(kernel='rbf',C=1e4,gamma =1e-9) #init Arousal
svr_case1_v = SVR(kernel='rbf',C=1e5,gamma =1e-10) #init Valence

#Predict Valence
a_v,df_v = predict(train_data_v,train_values_v,test_data_v,test_values_v,svr_case1_v)
#Predict Arousal
a_a,df_a = predict(train_data_a,train_values_a,test_data_a,test_values_a,svr_case1_a)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

(744, 36)


In [32]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [33]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 10000.0, 'gamma': 1e-09}
parameter Valence:	 {'C': 100000.0, 'gamma': 1e-10}


AROUSAL

In [36]:
df_a = df_a.round(2)
display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.28,5.49,6.94,3.49,3.49,4.38,2.68,5.91,3.3,4.99,...,6.6,5.16,4.79,4.78,5.24,4.64,5.74,6.26,4.17,3.08
1,6.20,5.90,6.40,3.10,3.50,3.60,3.00,5.80,3.6,6.20,...,7.0,5.00,3.70,5.80,4.70,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.65


VALENCE

In [37]:
df_v = df_v.round(2)
display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,6.08,5.64,6.19,4.35,4.73,4.91,4.01,5.65,4.16,5.35,...,6.22,5.79,4.72,5.04,5.57,5.06,5.41,5.33,4.29,4.01
1,2.70,6.40,6.00,2.70,5.00,3.20,4.60,4.30,4.10,6.70,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.22


# REGRESSION AND GRIDSEARCHCV

In [40]:
svr_a = SVR(kernel='rbf',C=1e4,gamma =1e-9)
svr_v = SVR(kernel='rbf',C=1e5,gamma =1e-10)

scores_a = regresion(svr_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_v,data,valence_mean) #Arousal Regression

In [41]:
print("R2 Score Arousal:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Arousal:	0.55 (+/- 0.09)
R2 Score Valence:	0.09 (+/- 0.21)


# CASE 4

In [45]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

svr_case1_a = SVR(kernel='rbf',C=1e2,gamma =1e-8) #init Arousal
svr_case1_v = SVR(kernel='rbf',C=1e2,gamma =1e-9) #init Valence

#Predict Valence
a_v,df_v = predict(train_data_v,train_values_v,test_data_v,test_values_v,svr_case1_v)
#Predict Arousal
a_a,df_a = predict(train_data_a,train_values_a,test_data_a,test_values_a,svr_case1_a)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

(744, 86)


In [43]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [44]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

parameter Arousal:	 {'C': 100.0, 'gamma': 1e-08}
parameter Valence:	 {'C': 100.0, 'gamma': 1e-09}


AROUSAL

In [46]:
df_a = df_a.round(2)
display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.44,4.73,6.69,3.48,3.5,4.99,2.89,6.03,3.29,5.16,...,6.4,5.26,4.96,4.77,5.4,4.33,5.75,6.45,3.91,2.95
1,6.20,5.90,6.40,3.10,3.5,3.60,3.00,5.80,3.60,6.20,...,7.0,5.00,3.70,5.80,4.7,4.90,4.80,6.30,3.30,2.20


R2 Score Arousal:	0.58


VALENCE

In [47]:
df_v = df_v.round(2)
display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,5.92,5.19,5.81,4.35,4.71,4.8,4.03,5.48,4.2,5.3,...,6.03,5.73,4.97,4.81,5.79,4.65,5.29,5.55,4.37,4.04
1,2.70,6.40,6.00,2.70,5.00,3.2,4.60,4.30,4.1,6.7,...,6.60,4.80,2.50,5.60,3.80,4.40,6.50,6.50,4.00,3.50


R2 Score Valence:	0.18


# REGRESSION AND GRIDSEARCHCV

In [48]:
svr_a = SVR(kernel='rbf',C=1e2,gamma =1e-8)
svr_v = SVR(kernel='rbf',C=1e2,gamma =1e-9)

scores_a = regresion(svr_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_v,data,valence_mean) #Arousal Regression

In [49]:
print("R2 Score Arousal:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

R2 Score Arousal:	0.51 (+/- 0.06)
R2 Score Valence:	0.06 (+/- 0.17)
